# Відправка повідомлення  в Redis на обробку вибраного файла і тут же його обобка модулем обробки повідомлень через окрему чергу, що не залучає контейнер оброкника **video-worker**

---
В даномій задачі Jupyter Notebooks  використовуються як інструменти тестування та відладки

In [1]:
import os
import json
import time
import pandas as pd
import matplotlib.pyplot as plt

import redis
from rq import Queue, Worker

# Читаю параметри підключення до Redis з env змінних
irds_host = os.getenv('RDS_HOST')
irds_port = os.getenv('RDS_PORT')
irds_psw = os.getenv('RDS_PSW')
# тут замінили на іншу чергу і в цій книзі її використовуємо
irdsq_outmsg = 'jupyter_queue' 

# для debug
#print(f"Прочита значення змінних підключеня до Redis {irds_host} {irds_port} {irds_psw} {irdsq_outmsg}")


## Підключаюся до  Redis

In [2]:
# Саме підключення
red = redis.StrictRedis(irds_host, irds_port, password=irds_psw, decode_responses=False)

# Пробую  Ping
rping=red.ping()
print( f"Статус підключення: {rping} ")

q_msg = None

# Аналізуємо значення ping
if rping:
    print("До Redis підключидлися успішно")
    q_msg = Queue( name=irdsq_outmsg, connection=red)
    print(f"Задач у черзі {irdsq_outmsg}: {len(q_msg)}")
else:
    print(" НE ПІДКЛЮЧИЛИСЯ ДО REDIS!!!")    

Статус підключення: True 
До Redis підключидлися успішно
Задач у черзі jupyter_queue: 0


## Пробуємо відправити повідмолення в Redis

In [3]:
#import json
print("Завантажую обробник черги....")
from utils import tasks
print("Завантаження виконано")
file_name="20260225-171507.avi"
try:
    # Публікуємо повідмолення в чергу для оборобки відое file_name
    message_o={"filename": file_name}
    message_s=json.dumps(message_o)
    print(f"Повідомлення для черги: {message_s}")
    job = q_msg.enqueue('utils.tasks.crttask_sendmsg',  message_s)
    print(f"Задачу додано в чергу: {job.id} для файлу {file_name}")
    # Список усіх ID завдань у черзі
    queued_job_ids = q_msg.job_ids
    print(f"ID завдань у черзі: {queued_job_ids}")

except Exception as e:
    print(f"Помилка черги Redis: {e}")

Завантажую обробник черги....
2026-02-25 17:51:40,891 - DEBUG - utils.tasks - debug message
2026-02-25 17:51:40,892 - DEBUG - utils.tasks - Модель завантажується з локального файлу: /usr/src/app/yolov8n.pt
2026-02-25 17:51:40,938 - DEBUG - utils.tasks - Модель успішно завантажено з локального файлу.
YOLOv8n summary: 129 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs
2026-02-25 17:51:41,010 - DEBUG - utils.tasks - YOLO model ready: (129, 3157200, 0, 8.8575488)
Завантаження виконано
Повідомлення для черги: {"filename": "20260225-171507.avi"}
Задачу додано в чергу: 2cb3c60e-dc51-4a71-ab71-fa0bd922b36a для файлу 20260225-171507.avi
ID завдань у черзі: ['2cb3c60e-dc51-4a71-ab71-fa0bd922b36a']


## Отримати статус тільки що відпавелного  завдання в чергу

In [ ]:
import time
# 1. Отримуємо ID завдання (з попередньої клітинки)
current_job_id = job.id 
print(f"Задач у черзі {irdsq_outmsg}: {len(q_msg)}")
print(f"Checking status for Job ID: {current_job_id}")
# Кількість ітерацій
i=3
# 2. Цикл очікування результату
while i!=0:
    # Оновлюємо дані про завдання з Redis
    job.refresh()
    
    status = job.get_status()
    print(f"Current status: {status}")

    if status == 'finished':
        print("Done! Result:", job.result)
        break
    elif status == 'failed':
        print("Job failed! Check worker logs.")
        # Можна вивести помилку: print(job.exc_info)
        break
    i -= 1
    time.sleep(2) # Чекаємо 2 секунди перед наступною перевіркою

## Обробка завдань з черги

In [ ]:
redde = redis.StrictRedis(irds_host, irds_port, password=irds_psw, decode_responses=False)
irds_queue=irdsq_outmsg
# Створюємо список об'єктів Queue правильно
# Для кожної назви черги в listen створюємо об'єкт Queue з підключенням
listen = [irds_queue]
queues = [Queue(irds_queue, connection=redde) for name in listen]

print(f"running worker for queues: {listen}")

try:
    print("Починаю роботу воркера")
    # Передаємо вже створені об'єкти черг у воркер
    # Параметр connection=red тут також бажано залишити
    worker = Worker(queues, connection=redde)
    process_result=worker.work(logging_level="DEBUG")
    print(f"Робота воркера завершена з результатом: {process_result}")

    # Список усіх ID завдань у черзі
    queued_job_ids = q_msg.job_ids
    print(f"ID завдань у черзі: {queued_job_ids}")

except Exception as e:
    print(f"Помилка під час роботи воркера: {e}")   


running worker for queues: ['jupyter_queue']
Починаю роботу воркера
17:52:00 Worker 1bc82151e2f449efa1c276894a25f6df: registering birth
17:52:00 Worker 1bc82151e2f449efa1c276894a25f6df: started with PID 556, version 2.7.0
17:52:00 Worker 1bc82151e2f449efa1c276894a25f6df: subscribing to channel rq:pubsub:1bc82151e2f449efa1c276894a25f6df
17:52:00 *** Listening on jupyter_queue...
17:52:00 Worker 1bc82151e2f449efa1c276894a25f6df: sent heartbeat to prevent worker timeout. Next one should arrive in 480 seconds.
17:52:00 Worker 1bc82151e2f449efa1c276894a25f6df: cleaning registries for queue: jupyter_queue
17:52:00 Worker 1bc82151e2f449efa1c276894a25f6df: *** Listening on jupyter_queue...
17:52:00 Worker 1bc82151e2f449efa1c276894a25f6df: sent heartbeat to prevent worker timeout. Next one should arrive in 480 seconds.
17:52:00 Worker 1bc82151e2f449efa1c276894a25f6df: dequeueing jobs on queues jupyter_queue and timeout 405
17:52:00 Worker 1bc82151e2f449efa1c276894a25f6df: dequeued job 2cb3c60e-